In [1]:
# from deepface import DeepFace
# import cv2
# import tensorflow as tf 
# import numpy,datetime,pygame,keyboard
# import xlwings as xw

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# # model = 'liveness.model'
# model = tf.keras.models.load_model(model)

In [3]:
# # xlwings
# # initialising workbook and worksheet
# workbook = xw.Book('attendance.xlsx')
# sheet_name = datetime.date.today().isoformat()

# try:
#     worksheet = workbook.sheets(sheet_name)
# except:
#     worksheet = workbook.sheets.add(sheet_name)

In [4]:
# worksheet.range('A1').value = 'STUDENT'
# worksheet.range('B1').value = 'DATE'
# worksheet.range('C1').value = 'TIME'

# students=[]

In [14]:
from deepface import DeepFace
import cv2
import tensorflow as tf 
import numpy,datetime,pygame,keyboard
import xlwings as xw

model = 'liveness.model'
model = tf.keras.models.load_model(model)

# xlwings
# initialising workbook and worksheet
workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet = workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)
    
worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

students=[]

# 0 for webcam(low quality), 1 for external cam
cap = cv2.VideoCapture(1)


# pygame
pygame.init()
pg_txt = (39,112,50)
pg_bg = (154,179,157)

font = cv2.FONT_HERSHEY_SIMPLEX
pygame_font = pygame.font.Font('freesansbold.ttf',32)

screen = pygame.display.set_mode((960,540))

background = pygame.image.load('school.jpeg')
background = pygame.transform.rotozoom(background,0,1.2)
screen.blit(background,(0,0))

# day today
t0 = datetime.date.today()
t0 = t0.day

running = True
s=2
i=0
entered=False
al_entered=False

while running:
    moment=datetime.datetime.now()
    hour=moment.hour
    minute=moment.minute
    second=moment.second
    day=moment.day
    month=moment.month
    year=moment.year
    
    date = f"{day}-{month}-{year}"
    time = f"{hour}.{minute}.{second}"
    
    screen.blit(background,(0,0))
    
    
    #check
    if day!=t0:
        t0 = day
        worksheet = workbook.sheets.add(date)
        
        worksheet.range('A1').value = 'STUDENT'
        worksheet.range('B1').value = 'DATE'
        worksheet.range('C1').value = 'TIME'
        students = []
        s=2
        i=0
        entered=False
        al_entered=False
        
        
        
    state, frame = cap.read()
    
    if(state!=True):
        break
    
    res = DeepFace.find(frame, db_path='./database', enforce_detection=False)
    avg=0
    cntr=1
    
    if len(res[0]['identity'])>0:
        
        name = res[0]['identity'][0].split('/')[1].split('\\')[1]
        xmin = int(res[0]['source_x'][0])
        ymin = int(res[0]['source_y'][0])
        
        w = res[0]['source_w'][0]
        h = res[0]['source_h'][0]
        
        xmax = int(xmin + w)
        ymax = int(ymin + h)
        
        img = frame[ymin:ymax, xmin:xmax]
        img = cv2.resize(img,(32,32))
        img = img.astype('float')/255.0
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = numpy.expand_dims(img,axis=0)
        
        liveness = model.predict(img)
        #this gives an arr of 2 vals conf_fake,conf_real,just find bigger one
        liveness = liveness[0].argmax()

        
        
        if liveness==1 and xmin!=0 and abs(xmax-xmin>100):
            print("hello")
            cv2.rectangle(frame, (xmin,ymin),(xmax,ymax), (0,255,0),1)
            cv2.rectangle(frame, (xmin,ymin-25),(xmax,ymin), (255,255,255),-1)    
            cv2.putText(frame, name, (xmin,ymin), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)     
            
            n_txt=pygame_font.render(name,True,pg_txt,pg_bg)
            n_txtrect=n_txt.get_rect()
            n_txtrect.center=(960-200,540//2)
            
            d_txt=pygame_font.render(date,True,pg_txt,pg_bg)
            d_txtrect=d_txt.get_rect()
            d_txtrect.center=(960-200,(540//2)+40)
            
            
            t_txt=pygame_font.render(time,True,pg_txt,pg_bg)
            t_txtrect=d_txt.get_rect()
            t_txtrect.center=(960-200,(540//2)+80)
            
            screen.blit(n_txt,n_txtrect)
            screen.blit(d_txt,d_txtrect)
            screen.blit(t_txt,t_txtrect)
            
            if keyboard.is_pressed('enter'):
                if(len(name)>0):
                    if name not in students:
                        worksheet.range(f'A{s}').value = name
                        worksheet.range(f'B{s}').value = date
                        worksheet.range(f'C{s}').value = time
                        students.append(name)
                        s=s+1
                        entered=True
                    else:
                        al_entered=True
            if i<15 and entered==True:
                e_txt=pygame_font.render('entered',True,pg_txt,pg_bg)
                e_txtrect=e_txt.get_rect()
                e_txtrect.center=(960-200,(540//2)+120)
                screen.blit(e_txt,e_txtrect)
                i+=1

            elif i<15 and name in students and al_entered==True:
                ae_txt=pygame_font.render('already entered',True,pg_txt,pg_bg)
                ae_txtrect=ae_txt.get_rect()
                ae_txtrect.center=(960-200,(540//2)+120)
                screen.blit(ae_txt,ae_txtrect)
                i+=1

            else:
                entered=False
                al_entered=False
                i=0
            
        image=pygame.image.frombuffer(frame.tostring(),frame.shape[1::-1],'BGR')
        image=pygame.transform.rotozoom(image,0,0.8)
        
        #round corners
        size=image.get_size()
        rect_img=pygame.Surface(size,pygame.SRCALPHA)
        pygame.draw.rect(rect_img,(255,255,255),(0,0,*size),border_radius=10)
        
        image=image.copy().convert_alpha()
        image.blit(rect_img,(0,0),None,pygame.BLEND_RGB_MIN)
        #round corner
        
        screen.blit(image,(20,80))
        
        
        
        pygame.display.update()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
#             print("xmin",xmin)
#             print("xmax",xmax)
            
#     cv2.imshow('attendance', frame)
    
#     c = cv2.waitKey(1)
    
#     if c==ord('q'):
#         break
    
cap.release()
pygame.quit()
# cv2.destroyAllWindows()

There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1556107997894287  seconds
1/1 [==============================] - 0s 99ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1664578914642334  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16826415061950684  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1755387783050537  seconds
1/1 [==============================] - 0s 18ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17337822914123535  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17311429977416992  seconds
1/1 [========================

find function lasts  0.1699512004852295  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16986894607543945  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16706633567810059  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16813254356384277  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16742992401123047  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1655871868133545  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  repre

find function lasts  0.18559694290161133  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16892790794372559  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.32287073135375977  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17266488075256348  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1800384521484375  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.169874906539917  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.

find function lasts  0.17468905448913574  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17610907554626465  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.32681846618652344  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.3262009620666504  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.3232080936431885  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17691302299499512  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.32826948165893555  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17833733558654785  seconds
1/1 [==========

There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17811226844787598  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.3262369632720947  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17465972900390625  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17188215255737305  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16966795921325684  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.16830945014953613  seconds
1/1 [==============================] - 0s 16ms/step
hello
The

find function lasts  0.17074275016784668  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17230868339538574  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.32676124572753906  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1766338348388672  seconds
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17112135887145996  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17337298393249512  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  

find function lasts  0.17220664024353027  seconds
1/1 [==============================] - 0s 17ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.32325267791748047  seconds
1/1 [==============================] - 0s 15ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17418408393859863  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1815185546875  seconds
1/1 [==============================] - 0s 16ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17331194877624512  seconds
1/1 [==============================] - 0s 17ms/step
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.17331647872924805  seconds
1/1 [==============================] - 0s 14ms/step
There are  5  representations found in  r

find function lasts  0.18589568138122559  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1899864673614502  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18400049209594727  seconds
1/1 [==============================] - 0s 17ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18277907371520996  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18482327461242676  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18531584739685059  seconds
1/1 [==============================] - 0s 16ms/step
hello
There ar

find function lasts  0.18890023231506348  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18802571296691895  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.19252562522888184  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18044495582580566  seconds
1/1 [==============================] - 0s 16ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.18634700775146484  seconds
1/1 [==============================] - 0s 15ms/step
hello
There are  5  representations found in  representations_vgg_face.pkl
find function lasts  0.1835775375366211  seconds
1/1 [==============================] - 0s 16ms/step
hello
There ar